In [1]:
import cv2
import numpy as np
import pyautogui
import time, os

kernel = np.ones((7,7),np.uint8)
cursor = [960,540]
pyautogui.FAILSAFE = False

In [2]:
def nothing(x):
    pass

# Distance between two centroids
def distance( c1, c2):
    distance = pow( pow(c1[0]-c2[0],2) + pow(c1[1]-c2[1],2) , 0.5)
    return distance

In [3]:
def calibration(colourName, Range, cap):

    name = 'Calibrate' + colourName
    cv2.namedWindow(name)

    cv2.createTrackbar('Hue',name,Range[0][0]+20,230, nothing)
    cv2.createTrackbar('Sat',name,Range[0][1],255, nothing)
    cv2.createTrackbar('Val',name,Range[0][2],255, nothing)

    while True:
        k = cv2.waitKey(5) & 0xFF
        if k == ord('d'):
            cv2.destroyWindow(name)
            return Range

        _, revframe = cap.read()
        frame = cv2.flip(revframe, 1)

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        hue = cv2.getTrackbarPos('Hue',name)
        sat = cv2.getTrackbarPos('Sat',name)
        val = cv2.getTrackbarPos('Val', name)

        lower = np.array([hue-20, sat, val])
        upper = np.array([hue+20, 255, 255])

        mask = cv2.inRange(hsv, lower, upper)
        eroded = cv2.erode(mask, kernel, iterations=1)
        dilated = cv2.dilate(eroded, kernel, iterations=1)

        cv2.imshow(name, dilated)
        if k == ord(' '):
            cv2.destroyWindow(name)
            return np.array([[hue-20, sat, val], [hue+10, 255, 255]])


In [4]:
def createMask(frame, colorRng):
    #cv2.inrange is used to filter out specific color then it undergoes erosion and dilation to generate mask

    mask = cv2.inRange(frame, colorRng[0], colorRng[1])

    #erosion = cv2.erode(mask, kernel, iterations=1)
    #dilation = cv2.dilate(erosion, kernel, iterations=1)
    opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    #return dilation
    return opening

def swap(array, i):
    array[0],array[i] = array[i],array[0]

def drawCentroid(frame, colorArea, mask, showCentroid, showContour):

    contour, _ = cv2.findContours( mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # gives size of contour
    l = len(contour)
    area = np.zeros(l)

    #Filtering contours based on Area
    for i in range(l):
        if cv2.contourArea(contour[i])>colorArea[0] and cv2.contourArea(contour[i])<colorArea[1]:
            area[i] = cv2.contourArea(contour[i])
        else:
            area[i] = 0
    # Sorted returns a copy insted of sorting in place
    decend = sorted(area, reverse = True)

    # bringing contours with largest valid area to the top which shall be later used to find centroid
    for i in range(l):
        if area[i] == decend[0]:
            swap( contour, i)

    if l > 0:
        # Finding centroid using method of moments
        M = cv2.moments(contour[0])

        if M['m00'] != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            center = (cx, cy)

            if showCentroid:
                cv2.circle(frame, center, 5, (0,0,255), -1)

            if showContour:
                cv2.drawContours(frame, contour[0], -1, (0,255,0), 5)

            return center

    else:
        return(-1,-1)   #return error handling values


In [5]:
def setCursor(yCenter, Yprev):
    yp = np.zeros(2)

    if abs(yCenter[0] - Yprev[0]) < 5 and abs(yCenter[1] - Yprev[1])<5:
        yp[0] = yCenter[0] + 0.9*(Yprev[0]-yCenter[0])
        yp[1] = yCenter[1] + 0.9*(Yprev[1]-yCenter[1])
    else:
        yp[0] = yCenter[0] + 0.1*(Yprev[0]-yCenter[0])
        yp[1] = yCenter[1] + 0.1*(Yprev[1]-yCenter[1])

    return yp

In [6]:
def chooseAction(yp, rc, bc):
    out = np.array(['move', 'false'])

    if rc[0]!=-1  and bc[0]!=-1:
        if distance(yp, rc)<50 and distance(yp,bc)<80 and distance(rc,bc)<50:
            # all centroids are close to one another
            #print(distance(yp, rc), distance(yp, bc), distance(rc, bc))
            out[0] = 'drag'
            out[1] = 'true'
            return out
        elif distance(rc,bc)<40:
            out[0] = 'left'
            return out
        elif distance(yp,rc)<40:
            out[0] = 'right'
            return out
        elif distance(yp,rc)>40 and rc[1]-bc[1]>150:
            out[0] = 'down'
            return out
        elif bc[1]-rc[1]>110:
            out[0] = 'up'
            return out
        elif distance(yp,bc)<50 and abs(rc[1]-yp[1]) > 110:
            out[0] = 'SS'
            #print(out[0], distance(yp,bc),abs(rc[1]-yp[1]) )
            return out
        else:
            return out
    else:
        out[0] = -1
        return out
    

In [7]:
# Defining default ranges for colour values
blue = np.array([[85,69,36],[128,255,255]])#  108,78,20-->105,69,36
yellow = np.array([[11,70,80],[61,255,255]])# 41,70,80--> 31,70,80
red = np.array([[155, 82, 72],[185, 255, 255]])# 174,85,72--> 175,82,58

# Area ranges for contours of different colours to be detected
r_area = [100,1500]
b_area = [100,1500]
y_area = [100,1700]

# Prior initialization of all centers
b_cen, y_pos, r_cen = [240,320],[240,320],[240,320]
cursor = [960,540]

# Status variables
perform = False
showCentroid = False
showContour = False

In [8]:
def changeStatus(key):
        global perform
        global showCentroid
        global showContour
        global yellow, red, blue

        if key == ord('p'):
                perform = not perform
                if perform:
                    print("Mouse simulation ON..")
                else:
                    print("Mouse simulation OFF")

        elif key == ord('c'):
                showCentroid = not showCentroid
                if showCentroid:
                    print("Showing Cenroid")
                else:
                    print("Not showing Centroid")

        elif key == ord('r'):
                print("              In recalibration mode.")
                print(" Use trackbars to calibrate press space when done")
                print("           Press D for default settings")
                print("=======================================================")

                # Calibration function present in functions.py
                yellow = calibration('Yellow', yellow, cap)
                blue = calibration('Blue', blue, cap)
                red = calibration('Red', red, cap)

        elif key == ord('d'):
                showContour = not showContour
                if showContour:
                    print("Showing Contours")
                else:
                    print("Not showing Contours")

        else:
            pass

In [9]:
def performAction(yp, rc, bc, action, drag, perform):
    if perform:
        cursor[0] = 4*(yp[0]-110)
        cursor[1] = 4*(yp[1]-110)
        if action == 'move':
            # if within boundary of frame go to respective cursor coordinates
            if yp[0]>110 and yp[0]<560 and yp[1]>110 and yp[1]<420:
                pyautogui.moveTo(cursor[0],cursor[1])

            # if on left edge
            elif yp[0]<110 and yp[1]>110 and yp[1]<420:
                pyautogui.moveTo( 8 , cursor[1])

            # if on right edge
            elif yp[0]>560 and yp[1]>110 and yp[1]<420:
                pyautogui.moveTo(1912, cursor[1])

            # if on top edge
            elif yp[0]>110 and yp[0]<560 and yp[1]<110:
                pyautogui.moveTo(cursor[0] , 8)

            # if on bottom edge
            elif yp[0]>110 and yp[0]<560 and yp[1]>420:
                pyautogui.moveTo(cursor[0] , 1072)

            # top left corner
            elif yp[0]<110 and yp[1]<110:
                pyautogui.moveTo(8, 8)

            # bottom left corner
            elif yp[0]<110 and yp[1]>420:
                pyautogui.moveTo(8, 1072)

            # bottom right corner
            elif yp[0]>560 and yp[1]>420:
                pyautogui.moveTo(1912, 1072)

            # top right corner
            else:
                pyautogui.moveTo(1912, 8)
                
        elif action == 'left':
            pyautogui.click(button = 'left')
            #time.sleep(0.3)

        elif action == 'right':
            pyautogui.click(button = 'right')
            time.sleep(0.3)

        elif action == 'up':
            pyautogui.scroll(5)

        elif action == 'down':
            pyautogui.scroll(-5)

        elif action == 'SS':
            os.system("gnome-screenshot --file=image.png")
            time.sleep(0.3)

        elif action == 'drag' and drag == 'true':
            global y_pos
            drag = 'false'
            pyautogui.mouseDown()

            while(1):

                k = cv2.waitKey(10) & 0xFF
                changeStatus(k)

                _, frameinv = cap.read()
                # flip horizontaly to get mirror image in camera
                frame = cv2.flip( frameinv, 1)

                hsv = cv2.cvtColor( frame, cv2.COLOR_BGR2HSV)

                b_mask = createMask( hsv, blue)
                r_mask = createMask( hsv, red)
                y_mask = createMask( hsv, yellow)

                Yprev = y_pos

                b_cen = drawCentroid( frame, b_area, b_mask, showCentroid, showContour)
                r_cen = drawCentroid( frame, r_area, r_mask, showCentroid, showContour)
                y_cen = drawCentroid( frame, y_area, y_mask, showCentroid, showContour)

                if 	Yprev[0]!=-1 and y_cen[0]!=-1:
                    y_pos = setCursor(y_cen, Yprev)

                performAction(y_pos, r_cen, b_cen, 'move', drag, perform)
                cv2.imshow('Frame', frame)

                # if fingers go far apart i.e, drag action stops then end
                if distance(y_pos,r_cen)>60 or distance(y_pos,b_cen)>60 or distance(r_cen,b_cen)>60:
                    break

            pyautogui.mouseUp()

In [10]:

cap = cv2.VideoCapture(0)
print("=======================================================")
print("              In calibration mode.")
print(" Use trackbars to calibrate press space when done")
print("           Press D for default settings")
print("=======================================================")

# Calibration ranges so as to detect only 3 colours
yellow = calibration('Yellow', yellow, cap)
blue = calibration('Blue', blue, cap)
red = calibration('Red', red, cap)
print("Calibration Successfull!!")

cv2.namedWindow('Frame')

print("=========================================================")
print("     press P to turn ON and OFF mouse control")
print("     press C to display centroid of various colours")
print("     press R to recalibrate")
print("     press D to show contours")
print("     Press ESC to exit")
print("=========================================================")

while (1):
    key = cv2.waitKey(10) & 0xFF
    # Checking status based on key pressed
    changeStatus(key)
    
    _, source = cap.read()
    frame = cv2.flip(source, 1)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    b_mask = createMask(hsv, blue)
    r_mask = createMask(hsv, red)
    y_mask = createMask(hsv, yellow)

    b_cen = drawCentroid( frame, b_area, b_mask, showCentroid, showContour)
    r_cen = drawCentroid( frame, r_area, r_mask, showCentroid, showContour)
    y_cen = drawCentroid( frame, y_area, y_mask, showCentroid, showContour)

    Yprev = y_pos

    if Yprev[0]!=-1 and y_cen[0]!=-1 and y_pos[0]!=-1:
        y_pos = setCursor(y_cen, Yprev)

    output = chooseAction(y_pos, r_cen, b_cen)
    if output[0]!=-1:
        performAction(y_pos, r_cen, b_cen, output[0], output[1], perform)

    cv2.imshow('Frame', frame)

    if key == 27:
        break
cv2.destroyAllWindows()

              In calibration mode.
 Use trackbars to calibrate press space when done
           Press D for default settings
Calibration Successfull!!
     press P to turn ON and OFF mouse control
     press C to display centroid of various colours
     press R to recalibrate
     press D to show contours
     Press ESC to exit
Showing Cenroid
Mouse simulation ON..
Mouse simulation OFF
Not showing Centroid
Showing Cenroid
Not showing Centroid
Showing Cenroid
Not showing Centroid


Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forci

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forci

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forci

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forci

...

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forci

Mouse simulation OFF


Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forci

Mouse simulation ON..


Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Sankruti\anaconda3\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forci